In [1]:
import pandas as pd
import random

In [2]:
start = 1
end = 637231
random.seed(42)
# Calculate the size of the desired subset (20% of the range)
subset_size = int(0.08 * (end - start + 1))

# Generate a random subset of that size
random_subset = random.sample(range(start, end + 1), subset_size)

In [3]:
interactions = pd.read_csv("data/100k_dataset/100k_dataset.inter_original", sep="\t")

In [4]:
tracks_df = pd.read_csv('data/100k_dataset/tracks.txt.gz_original', sep='\t', compression='gzip', header=None)
users_df = pd.read_csv('data/100k_dataset/demo.txt.gz_original', sep='\t', compression='gzip', header=None)


In [5]:
new_item_list = tracks_df.loc[sorted(random_subset)]

In [6]:
new_item_list = new_item_list[new_item_list[2].map(new_item_list[2].value_counts()) >= 300]
valid_countries_set = set(new_item_list[2])

In [7]:
index_translation = {old_index:new_index for new_index, old_index in enumerate(new_item_list.index)}

In [8]:
filtered_interactions = interactions[interactions['item_id:token'].isin(set(new_item_list.index))]
filtered_interactions['item_id:token'] = filtered_interactions['item_id:token'].map(index_translation)

C:\Users\Martin\AppData\Local\Temp\ipykernel_17396\109829266.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_interactions['item_id:token'] = filtered_interactions['item_id:token'].map(index_translation)


In [9]:
interaction_counts = filtered_interactions["user_id:token"].value_counts()
user_ids_less_than_20_interactions = set(interaction_counts[interaction_counts < 20].index)

In [10]:
users_in_underrepresented_countries = set(users_df[~users_df[0].isin(valid_countries_set)].index)

In [11]:
users_to_keep = sorted(list(set(users_df.index).intersection(set(filtered_interactions["user_id:token"].values)) - user_ids_less_than_20_interactions - users_in_underrepresented_countries))
user_id_map = {old_id : new_id for new_id, old_id in enumerate(users_to_keep)}
filtered_interactions = filtered_interactions[filtered_interactions["user_id:token"].isin(users_to_keep)]
filtered_interactions['user_id:token'] = filtered_interactions['user_id:token'].map(user_id_map)

In [12]:
filtered_interactions

,user_id:token,item_id:token
10,0,0
38,0,1
54,0,2
55,0,3
63,0,4
...,...,...
14639101,6452,32088
14639109,6452,33373
14639120,6452,35990
14639133,6452,37021


In [13]:
fi_value_counts = filtered_interactions["item_id:token"].value_counts()


filtered_interactions_filtered = filtered_interactions[filtered_interactions["item_id:token"].isin(fi_value_counts[fi_value_counts >= 4].index)]
filtered_item_indices = set(filtered_interactions_filtered["item_id:token"].value_counts().index)

In [14]:
index_translation2 = {old_index:new_index for new_index, old_index in enumerate(sorted(filtered_item_indices))}

In [15]:
filtered_interactions_filtered['item_id:token'] = filtered_interactions_filtered['item_id:token'].map(index_translation2)

C:\Users\Martin\AppData\Local\Temp\ipykernel_17396\1768535217.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_interactions_filtered['item_id:token'] = filtered_interactions_filtered['item_id:token'].map(index_translation2)


In [16]:
new_item_list = new_item_list.reset_index(drop=True)

In [17]:
new_item_list = new_item_list[new_item_list.index.isin(filtered_item_indices)].reset_index(drop=True)

In [18]:
filtered_users = users_df.loc[users_to_keep]

In [19]:
filtered_interactions_filtered.to_csv("data/100k_dataset/100k_dataset.inter", sep="\t", index=False)
new_item_list.to_csv('data/100k_dataset/tracks.txt.gz', sep="\t", index=False, compression="gzip", header=None)
filtered_users.to_csv('data/100k_dataset/demo.txt.gz', sep="\t", index=False, compression="gzip", header=None)

In [20]:
new_item_list.index

RangeIndex(start=0, stop=42487, step=1)

In [21]:
(filtered_interactions_filtered["item_id:token"] == 34522).value_counts()

item_id:token
False    769023
True         10
Name: count, dtype: int64

In [22]:
len(set(filtered_interactions_filtered["item_id:token"]))

42487